In [1]:
import pandas as pd
import requests as r
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
import plotly.express as px
from utils.settings import DETERMINISTIC_MODELS, DETERMINISTIC_VARS

In [5]:
def get_historical_data(latitude=53.55,
                        longitude=9.99,
                        variables='temperature_2m',
                        timezone='auto',
                        model='best_match',
                        start_date='1991-01-01',
                        end_date='2020-12-31'):
    """
    Get historical data for a point
    """
    payload = {
        "latitude": latitude,
        "longitude": longitude,
        "hourly": variables,
        "timezone": timezone,
        "models": model,
        "start_date": start_date,
        "end_date": end_date
    }

    resp = r.get("https://archive-api.open-meteo.com/v1/archive",
                 params=payload)
    data = pd.DataFrame.from_dict(resp.json()['hourly'])
    data['time'] = pd.to_datetime(
        data['time'], format='%Y-%m-%dT%H:%M')

    data = data.dropna()

    return data

def compute_daily_stats(latitude=53.55, longitude=9.99, model='era5',
                        start_date='1991-01-01', end_date='2020-12-31'):
    """Starting from hourly data compute daily aggregations which are
    useful for other functions that compute climatologies"""

    df = get_historical_data(
        variables='temperature_2m,precipitation,snowfall,cloudcover,windspeed_10m',
        latitude=latitude,
        longitude=longitude,
        timezone='GMT',  # Use UTC to avoid issues with daily computations
        model=model,
        start_date=start_date,
        end_date=end_date)

    daily = df.resample('1D', on='time').agg(t2m_mean=('temperature_2m', 'mean'),
                                             t2m_min=('temperature_2m', 'min'),
                                             t2m_max=('temperature_2m', 'max'),
                                             wind_speed_max=(
                                                 'windspeed_10m', 'max'),  # in km/h
                                             daily_rain=(
                                                 'precipitation', 'sum'),  # in mm
                                             daily_snow=(
                                                 'snowfall', 'sum'),  # in cm
                                             cloudcover_mean=(
                                                 'cloudcover', 'mean')  # in %
                                             )

    return daily

def get_historical_daily_data(latitude=53.55,
                        longitude=9.99,
                        variables='precipitation_sum',
                        timezone='GMT',
                        model='best_match',
                        start_date='1991-01-01',
                        end_date='2020-12-31'):
    """
    Get historical data for a point
    """
    payload = {
        "latitude": latitude,
        "longitude": longitude,
        "daily": variables,
        "timezone": timezone,
        "models": model,
        "start_date": start_date,
        "end_date": end_date
    }

    resp = r.get("https://archive-api.open-meteo.com/v1/archive",
                 params=payload)
    data = pd.DataFrame.from_dict(resp.json()['daily'])
    data['time'] = pd.to_datetime(
        data['time'], format='%Y-%m-%dT%H:%M')

    data = data.dropna()

    return data

In [98]:
daily = get_historical_daily_data(start_date='1981-01-01',
                                  end_date=(pd.to_datetime('now',utc=True)-pd.to_timedelta('1 day')).strftime("%Y-%m-%d"),
                                  variables='precipitation_sum',)

In [99]:
var = 'precipitation_sum'
current_year = pd.to_datetime('now', utc=True).year

daily = daily[~((daily.time.dt.month == 2) & (daily.time.dt.day == 29))]
daily[f'{var}_yearly_acc'] = daily.groupby(daily.time.dt.year)[var].transform(lambda x: x.cumsum())

quantiles = daily.groupby([daily.time.dt.day, daily.time.dt.month])[f'{var}_yearly_acc'].quantile(q=0.05).to_frame().rename(columns={f'{var}_yearly_acc':'q1'})
quantiles['q2'] = daily.groupby([daily.time.dt.day, daily.time.dt.month])[f'{var}_yearly_acc'].quantile(q=0.5)
quantiles['q3'] = daily.groupby([daily.time.dt.day, daily.time.dt.month])[f'{var}_yearly_acc'].quantile(q=0.95)

quantiles.index.set_names(["day", "month"], inplace=True)
quantiles.reset_index(inplace=True)
quantiles['dummy_date'] = pd.to_datetime(f'{current_year}-' + quantiles.month.astype(str) + "-" + quantiles.day.astype(str))
quantiles.sort_values(by='dummy_date', inplace=True)

daily = daily[daily.time.dt.year==current_year].merge(quantiles, left_on='time', right_on='dummy_date', how='right')

In [132]:
fig = make_subplots()
traces = []

fig.add_trace(
    go.Scatter(
        x=daily.dummy_date,
        y=daily['q1'],
        mode='lines',
        name='5th Percentile',
        line=dict(width=.1, color='gray'),
        showlegend=False,),
)

fig.add_trace(
    go.Scatter(
        x=daily.dummy_date,
        y=daily['q3'],
        mode='lines',
        name='5-95th percentiles range',
        line=dict(width=.1, color='gray'),
        showlegend=True,
        fill='tonexty', ),
        
)

fig.add_trace(
    go.Scatter(
        x=daily.dummy_date,
        y=daily['precipitation_sum_yearly_acc'],
        mode='lines',
        name=current_year,
        line=dict(width=3, color='black'),
        showlegend=True,),
)
fig.add_trace(
    go.Scatter(
        x=[daily.time.max()],
        y=[daily['precipitation_sum_yearly_acc'].max()],
        mode='markers',
        name='Latest',
        marker=dict(size=15, color='black'),
        showlegend=False,),
)

fig.add_trace(
    go.Scatter(
        x=daily.dummy_date,
        y=daily['q2'],
        mode='lines',
        name='50th Percentile',
        line=dict(width=1,color='gray',dash='dash'),
        showlegend=True,),
)

fig.update_layout(
    margin={"r": 0.1, "t": 0.1, "l": 0.1, "b": 0.1},
    template='plotly_white',
    barmode='stack',
    legend=dict(orientation='h'),
    yaxis=dict(showgrid=True),
)



In [122]:
daily.time.max()

Timestamp('2023-09-21 00:00:00')

In [112]:
daily['dummy_date'].iloc[-1]

Timestamp('2023-12-31 00:00:00')

In [78]:
stats = compute_monthly_clima()

In [96]:
end_date = (pd.to_datetime('now',utc=True)-pd.to_timedelta('2 days')).strftime("%Y-%m-%d")
start_date = pd.to_datetime('now',utc=True).replace(month=1, day=1).strftime("%Y-%m-%d")
stats_year = compute_monthly_clima(start_date=start_date, end_date=end_date)

In [7]:
def get_forecast_data(latitude=53.55,
                      longitude=9.99,
                      variables=",".join(DETERMINISTIC_VARS),
                      timezone='auto',
                      model=DETERMINISTIC_MODELS[0],
                      forecast_days = 7,
                      from_now=True):
    payload = {
        "latitude": latitude,
        "longitude": longitude,
        "daily": variables,
        "timezone": timezone,
        "models": model,
        "forecast_days": forecast_days
    }

    resp = r.get("https://api.open-meteo.com/v1/forecast",
                 params=payload)
    data = pd.DataFrame.from_dict(resp.json()['daily'])
    data['time'] = pd.to_datetime(
        data['time']).dt.tz_localize(resp.json()['timezone'])

    # Optionally subset data to start only from previous hour
    if from_now:
        data = data[data.time >= pd.to_datetime(
            'now', utc=True).tz_convert(resp.json()['timezone']).floor('H')]

    return data

In [14]:
df = get_forecast_data(model="ecmwf_ifs04,gfs_seamless,jma_seamless,icon_seamless,gem_seamless,meteofrance_seamless",
                       longitude=22.9425,
                       latitude=39.361,
                       variables='precipitation_sum,precipitation_hours',
                       forecast_days=7,
                       from_now=False)

In [20]:
df = df.set_index('time')

In [45]:
fig = df.loc[:,df.columns.str.contains('precipitation_sum')].cumsum().plot(backend='plotly')
# fig.add_trace(
#     go.Scatter(
#         x=df.index,
#         y=df.loc[:,df.columns.str.contains('precipitation_hours')].values,
#         # yaxis="y2"
#     ))
fig. update_layout(showlegend=False)

In [48]:
def make_lineplot_timeseries(df, var, mode='lines+markers'):
    traces = []
    for col in df.columns[df.columns.str.contains(var)]:
        # First do the first 48 hrs
        traces.append(
            go.Scatter(
                x=df.loc[:, 'time'],
                y=df.loc[:, col],
                mode=mode,
                name=col,
                marker=dict(size=5),
                line=dict(width=2),
                showlegend=False),
        )

    return traces


def make_barplot_timeseries(df, var):
    traces = []
    for col in df.columns[df.columns.str.contains(var)]:
        traces.append(
            go.Bar(
                x=df['time'],
                y=df[col],
                name=col,
                opacity=0.6,
                marker=dict(color='rgb(26, 118, 255)'),
                showlegend=False),
        )

    return traces

def make_subplot_figure(data):
    traces_temp = make_lineplot_timeseries(data, 'temperature_2m')
    traces_precipitation = make_barplot_timeseries(data, 'precipitation')
    traces_wind = make_lineplot_timeseries(data, 'windspeed_10m')
    traces_cloud = make_lineplot_timeseries(data, 'cloudcover', mode='markers')

    fig = make_subplots(
        rows=4,
        cols=1,
        shared_xaxes=True,
        vertical_spacing=0.05,
        row_heights=[0.5, 0.4, 0.3, 0.3])

    for trace_temp in traces_temp:
        fig.add_trace(trace_temp, row=1, col=1)
    for trace_precipitation in traces_precipitation:
        fig.add_trace(trace_precipitation, row=2, col=1)
    for trace_wind in traces_wind:
        fig.add_trace(trace_wind, row=3, col=1)
    for trace_cloud in traces_cloud:
        fig.add_trace(trace_cloud, row=4, col=1)

    fig.update_layout(
        xaxis=dict(showgrid=True,
                   range=[data['time'].min(),
                          data['time'].max()]),
        yaxis=dict(showgrid=True,),
        height=800,
        margin={"r": 0.1, "t": 0.1, "l": 0.1, "b": 0.1},
        template='plotly_white',
        barmode='overlay'
    )

    fig.update_yaxes(title_text="2m Temp [°C]", row=1, col=1)
    fig.update_yaxes(title_text="Prec. [mm]", row=2, col=1)
    fig.update_yaxes(title_text="Wind. [kph]", row=3, col=1)
    fig.update_yaxes(title_text="Cloud cover [%]", row=4, col=1)
    fig.update_yaxes(showgrid=True, gridwidth=4)
    fig.update_xaxes(minor=dict(ticks="inside", showgrid=True,
                     gridwidth=3), showgrid=True, gridwidth=4)

    return fig


make_subplot_figure(df)